In [ ]:
from trianglechain import TriangleChain
from cosmic_toolbox import arraytools as at, colors
import legacy_abc
from legacy_abc.analysis import ctx
import matplotlib.pyplot as plt
import os

# Note: To run this notebook you need to additionally install
# - galsbi (see https://cosmo-docs.phys.ethz.ch/galsbi/ if needed)
# - SExtractor (https://sextractor.readthedocs.io/en/latest/)

colors.set_cycle()
C = colors.get_colors()
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": ["Computer Modern Roman"],
    }
)

title_size = 20
label_size = 16
tick_size = 12

figsize = 15

In [ ]:
import ivy
import numpy as np

def create_ctx():
    ctx = ivy.context.create_ctx(
        parameters=ivy.load_configs("ufig.config.common")
    )
    
    ctx.parameters.catalog_precision=np.float32
    ctx.parameters.size_x = 100
    ctx.parameters.size_y = 100
    ctx.parameters.seeing = 0.7
    
    ctx.numgalaxies = 3
    ctx.galaxies = ivy.context.create_ctx(
        x=np.array([10, 40, 70]),
        y=np.array([30, 80, 20]),
        nphot=np.array([1e4, 1e5, 1e3]),
        r50=np.array([2, 4, 1]),
        sersic_n=np.array([1, 2, 3]),
        e1=np.array([-0.1, 0.1, 0]),
        e2=np.array([-0.5, 0.3, 0.2]),
        psf_beta=2,
        psf_e1=np.array([0.0, 0.0, 0.0]),
        psf_e2=np.array([0.0, 0.0, 0.0]),
        psf_f1=np.array([0.0, 0.0, 0.0]),
        psf_f2=np.array([0.0, 0.0, 0.0]),
        psf_g1=np.array([0.0, 0.0, 0.0]),
        psf_g2=np.array([0.0, 0.0, 0.0]),
        psf_kurtosis=np.array([0.0, 0.0, 0.0]),
        
    )
    
    ctx.numstars = 3
    ctx.stars = ivy.context.create_ctx(
        x=np.array([30, 90, 20]),
        y=np.array([10, 50, 50]),
        nphot=np.array([1e5, 1e4, 1e3]),
        mag=np.array([35,35,35]), # dummy
        psf_beta=2,
        psf_e1=np.array([0.0, 0.0, 0.0]),
        psf_e2=np.array([0.0, 0.0, 0.0]),
        psf_f1=np.array([0.0, 0.0, 0.0]),
        psf_f2=np.array([0.0, 0.0, 0.0]),
        psf_g1=np.array([0.0, 0.0, 0.0]),
        psf_g2=np.array([0.0, 0.0, 0.0]),
        psf_kurtosis=np.array([0.0, 0.0, 0.0]),
        
    )
    
    ctx.image = np.zeros((ctx.parameters.size_x, ctx.parameters.size_y))
    ctx.current_filter="i"
    return ctx

In [ ]:
from astropy.io import fits
from astropy.visualization import ImageNormalize, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import PercentileInterval
import matplotlib.pyplot as plt
from ufig.plugins import render_galaxies_flexion, render_stars_photon, add_psf, background_noise
ctx = create_ctx()
add_psf.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()

interval = PercentileInterval(99)
vmin, vmax = interval.get_limits(ctx.image)
norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())

fig, axs = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)

index = 0
axs[index].set_title("seeing: 0.5 arcsec", fontsize=title_size)
ctx = create_ctx()
ctx.parameters.seeing = 0.5
add_psf.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()
axs[index].imshow(ctx.image, cmap='gray', norm=norm)
axs[index].set_xticks([])
axs[index].set_yticks([])

index = 1
axs[index].set_title("seeing: 1 arcsec", fontsize=title_size)
ctx = create_ctx()
ctx.parameters.seeing = 1
add_psf.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()
axs[index].imshow(ctx.image, cmap='gray', norm=norm)

index=2
axs[index].set_title(r"seeing: 3 arcsec", fontsize=title_size)
ctx = create_ctx()
ctx.parameters.seeing = 3
add_psf.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()
axs[index].imshow(ctx.image, cmap='gray', norm=norm)
fig.savefig("psf_variations.png", transparent=True,bbox_inches="tight", dpi=600)

In [ ]:
ctx = create_ctx()
ctx.parameters.background_sigma = 1
add_psf.Plugin(ctx)()
background_noise.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()

interval = PercentileInterval(99)
vmin, vmax = interval.get_limits(ctx.image)
norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())

fig, axs = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)

index = 0
axs[index].set_title("low noise", fontsize=title_size)
ctx = create_ctx()
ctx.parameters.background_sigma = 1
add_psf.Plugin(ctx)()
background_noise.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()
axs[index].imshow(ctx.image, cmap='gray', norm=norm)
axs[index].set_xticks([])
axs[index].set_yticks([])

index = 1
axs[index].set_title("higher noise", fontsize=title_size)
ctx.parameters.background_sigma = 5
ctx = create_ctx()
add_psf.Plugin(ctx)()
background_noise.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()
axs[index].imshow(ctx.image, cmap='gray', norm=norm)

index=2
axs[index].set_title("varying noise", fontsize=title_size)
ctx = create_ctx()
bkg_rms = np.zeros_like(ctx.image)
bkg_rms[50:, 50:] = 1
bkg_rms[:50, :50] = 10
bkg_rms[50:, :50] = 3
bkg_rms[:50, 50:] = 3
ctx.parameters.background_type = "map"
ctx.parameters.bkg_rms_file_name = "bkg_rms.h5"
ctx.parameters.maps_remote_dir = os.getcwd()
at.save_hdf("bkg_rms.h5", bkg_rms)
add_psf.Plugin(ctx)()
background_noise.Plugin(ctx)()
render_galaxies_flexion.Plugin(ctx)()
render_stars_photon.Plugin(ctx)()
axs[index].imshow(ctx.image, cmap='gray', norm=norm)
fig.savefig("bkg_variations.png", transparent=True, bbox_inches="tight", dpi=600)
os.remove("bkg_rms.h5")

In [ ]:
from galsbi import GalSBI
from astropy.visualization import ZScaleInterval
model = GalSBI("Fischbacher+24")
model(mode="image")

images = model.load_images()

interval = PercentileInterval(99)
vmin, vmax = interval.get_limits(images["image i"])
norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())

fig, axs = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)

index = 0
axs[index].set_title(r"$r$", fontsize=title_size)
axs[index].imshow(images["image r"], cmap='gray', norm=norm)
axs[index].set_xticks([])
axs[index].set_yticks([])

index = 1
axs[index].set_title(r"$i$", fontsize=title_size)
axs[index].imshow(images["image i"], cmap='gray', norm=norm)
axs[index].set_xticks([])
axs[index].set_yticks([])

index=2
axs[index].set_title(r"$z$", fontsize=title_size)
axs[index].imshow(images["image z"], cmap='gray', norm=norm)
axs[index].set_xticks([])
axs[index].set_yticks([])

for f in ["g", "r", "i", "z", "y"]:
    os.remove(f"GalSBI_sim_0_{f}_image.fits")
    os.remove(f"GalSBI_sim_0_{f}_ucat.gal.cat")
fig.savefig("galsbi.png", transparent=True, bbox_inches="tight", dpi=600)

In [ ]:
from cosmic_toolbox import arraytools as at
from trianglechain import TriangleChain
from cosmic_toolbox import colors
import sys

# Note: This cell creates a very large image which requires a lot of RAM. 
# You can change the image size by modifying the parameters size_x and size_y in
# docs/notebooks/sextractor_config.py

colors.set_cycle()

sys.path.append("../docs/notebooks/")
ctx = ivy.execute("sextractor_config")
cat = at.load_hdf_cols("ufig_r_forced_photo.sexcat")
os.remove("ufig_r_forced_photo.sexcat")

In [ ]:
tri = TriangleChain(
    params=["MAG_AUTO", "FLUX_RADIUS", "ELLIPTICITY"],
    labels=[
        r"\texttt{MAG\_AUTO}",
        r"\texttt{FLUX\_RADIUS}",
        r"\texttt{ELLIPTICITY}",
    ],
    ranges={"MAG_AUTO": [14, 27], "FLUX_RADIUS": [1,10], "ELLIPTICITY": [0,1]},
    histograms_1D_density=False,
    fill=True,
    size=3,
    grid_kwargs={"fontsize_ticklabels": 20}
)
tri.contour_cl(cat, label="all objects")
tri.contour_cl(cat[cat["CLASS_STAR"]>0.9], label="stars", show_legend=True)
plt.savefig("catalog.png", transparent=True, bbox_inches="tight", dpi=600)